In [432]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np

Training data 로드

In [434]:
dataset=pd.read_csv('./gdrive/My Drive/datamining/train.csv')
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


이름을 통해 성별 및 직업 추출하여 숫자로 변환

In [0]:
def convertName(data):
  
  for i in range(len(data)):
    curName=data["Name"][i].split(",")[1].split(".")[0].strip()
    
    if data["Age"][i]==None:
      print("None")
      
    if curName == "Mr":
      data["Name"][i]=0
      data["Sex"][i]=0
    
    elif(curName=="Miss" or curName=="Ms" or curName=="Mme" or curName=="Mlle" or curName=="Mrs"):
      data["Name"][i]=1
      data["Sex"][i]=1

    elif(curName=="Master"):
      data["Name"][i]=2
      
    elif(curName=="Dr"):
      data["Name"][i]=3

    else:
      data["Name"][i]=4
      # X["Sex"][i]=0

  return data


Ensemble 모델 반환

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV

def makeEnsembleModel(X_train,Y_train):
  
 
  #model1 - DecisionTreeClassifier
  scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

  gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                    param_grid={'min_samples_split': range(2, 403, 10)},
                    scoring=scoring, refit='AUC', return_train_score=True)
  gs.fit(X_train,Y_train)
  model1=gs
  print("model 1: ", model1.best_score_)

  #model2 - LinearSVC
  pipe = Pipeline([
      # the reduce_dim stage is populated by the param_grid
      ('reduce_dim', 'passthrough'),
      ('classify', LinearSVC(dual=False, max_iter=10000))
  ])

  N_FEATURES_OPTIONS = [2, 4, 8]
  C_OPTIONS = [1, 10, 100, 1000]
  param_grid = [
      {
          'reduce_dim': [PCA(iterated_power=7), NMF()],
          'reduce_dim__n_components': N_FEATURES_OPTIONS,
          'classify__C': C_OPTIONS
      },
      {
          'reduce_dim': [SelectKBest(chi2)],
          'reduce_dim__k': N_FEATURES_OPTIONS,
          'classify__C': C_OPTIONS
      },
  ]
  reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

  grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
  grid.fit(X_train, Y_train)
  model2=grid
  print("model2: ", model2.best_score_)


  #model3 - ExtraTreesClassifier
  ExtC = ExtraTreesClassifier()

  ex_param_grid = {"max_depth": [None],
                "max_features": [1, 3, 10],
                "min_samples_split": [2, 3, 10],
                "min_samples_leaf": [1, 3, 10],
                "bootstrap": [False],
                "n_estimators" :[100,300],
                "criterion": ["gini"]}


  gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, scoring="accuracy", n_jobs= 4, verbose = 1)

  gsExtC.fit(X_train,Y_train)
  model3= gsExtC
  print("model3: ",model3.best_score_)

  ensemble = VotingClassifier(estimators=[('model1',model1),('model2',model2),('model3',model3)])
  # ensemble.fit(X_train,Y_train)

  return ensemble

In [437]:
len(X)

891

Training data - X, Y 추출

In [438]:
X=dataset.drop(columns=['PassengerId','Cabin','Ticket'])
repCol3 = {  "male":0, "female" : 1}
repCol8 = {"C" : 0 ,   "Q" : 1 , 'S' : 2  }
X.replace({"Sex": repCol3, "Embarked": repCol8} , inplace = True )
X['Embarked'].fillna(2,inplace=True)
X=convertName(X)
Y=X.pop("Survived")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Age 값이 Null이 아닌 data 추출

In [0]:
idx_NaN_age=list(X["Age"][X["Age"].notnull()].index)
Age_X=[]

for i in idx_NaN_age:
  Age_X.append(X.iloc(0)[i])

Age_X=pd.DataFrame(Age_X)
Age_Y=Age_X.pop("Age")
#Age_Y=pd.DataFrame(Age_Y)

Age 값이 Null인 data 추출

In [440]:
idx_age=list(X["Age"][X["Age"].isnull()].index)
Age_NaN_X=[]
for i in idx_age:
  Age_NaN_X.append(X.iloc(0)[i])

Age_NaN_X=pd.DataFrame(Age_NaN_X)
Age_NaN_X.pop("Age")

5     NaN
17    NaN
19    NaN
26    NaN
28    NaN
       ..
859   NaN
863   NaN
868   NaN
878   NaN
888   NaN
Name: Age, Length: 177, dtype: float64

Age 값 예측하는 모델 training 시킴

Age가 Null인 값들 예측 후 채워서 반환

In [0]:
def fillNaNAge(ensemble,input):
  idx_age=list(input["Age"][input["Age"].isnull()].index)
  Age_NaN_X=[]
  for i in idx_age:
    Age_NaN_X.append(input.iloc(0)[i])

  Age_NaN_X=pd.DataFrame(Age_NaN_X)
  Age_NaN_X.pop("Age")

  Age_predict=ensemble.predict(Age_NaN_X)

  idx=0
  for i in idx_age:
    input["Age"][i]=Age_predict[idx]
    idx+=1

  return input;

Mondrian Forest 알고리즘을 사용하여 age 예측 - https://github.com/scikit-garden/scikit-garden

In [442]:
pip install scikit-garden

In [443]:
from skgarden import MondrianForestClassifier
from skgarden import MondrianForestRegressor
mfr=MondrianForestRegressor(random_state=1,max_depth=5)
mfr.fit(Age_X,Age_Y)

# Normalization
X["Fare"]=(X["Fare"]-X["Fare"].min())/(X["Fare"].max()-X["Fare"].min())
X["Age"]=(X["Age"]-X["Age"].min())/(X["Age"].max()-X["Age"].min())

# Age 예측
X=fillNaNAge(mfr,X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
#RandomForestRegressor를 사용하여 이름 예측
# from sklearn.ensemble import RandomForestRegressor

# regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
# regressor.fit(Age_X,Age_Y)
# X=fillNaNAge(regressor,X)

In [0]:
 X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.7, random_state=1)

In [0]:

# # model1 = DecisionTreeClassifier(max_depth=10, random_state=0).fit(X_train, Y_train)
# # model2 = BaggingClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=0).fit(X_train, Y_train)
# # model3 = 

# model1 = DecisionTreeClassifier(max_depth=10, random_state=0)
# #model2 = BaggingClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=0)
# model2 = GaussianNB()
# #model3 = svm.SVC()
# model3=LinearSVC(C=1)
# model4 = SGDClassifier()

# ensemble = VotingClassifier(estimators=[('model1',model1),('model2',model2),('model3',model3),('model4',model4)])

In [447]:
# ensemble=makeEnsembleModel(X_train,Y_train)
# history=ensemble.fit(X_train,Y_train)
# scores=ensemble.score(X_test,Y_test)
# scores

ensemble=makeEnsembleModel(X,Y)
history=ensemble.fit(X,Y)

model 1:  0.8567063492063494
model2:  0.7979035639412999
Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   33.3s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   39.0s finished


model3:  0.8316561844863731
Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   33.1s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   38.8s finished


0.8157051282051282

Test data 추출 및 데이터 변환


In [448]:
test=pd.read_csv("./gdrive/My Drive/datamining/test.csv")
test_x=test.drop(columns=['PassengerId','Cabin','Ticket'])
test_x=convertName(test_x)
test_x.replace({"Sex": repCol3, "Embarked": repCol8} , inplace = True )
# Normalization
test_x["Fare"]=(test_x["Fare"]-test_x["Fare"].min())/(test_x["Fare"].max()-test_x["Fare"].min())
test_x["Age"]=(test_x["Age"]-test_x["Age"].min())/(test_x["Age"].max()-test_x["Age"].min())
test_x=fillNaNAge(mfr,test_x)
test_x['Fare'].fillna(test.Fare.mean(),inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [449]:
test_pred=ensemble.predict(test_x)
test_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [0]:
submission=pd.DataFrame({
    "PassengerId":test["PassengerId"],
    "Survived":test_pred
})
submission.to_csv("./gdrive/My Drive/datamining/titanic prediction.csv",index=False)

In [0]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.metrics import make_scorer
# from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC
# from sklearn.decomposition import PCA, NMF
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.model_selection import GridSearchCV

# def makeEnsembleModel(X_train,Y_train):
  
 
#   #model1 - DecisionTreeClassifier
#   scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

#   gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
#                     param_grid={'min_samples_split': range(2, 403, 10)},
#                     scoring=scoring, refit='AUC', return_train_score=True)
#   gs.fit(X_train,Y_train)
#   model1=gs
#   print("model 1: ", model1.best_score_)
#   #model1 = DecisionTreeClassifier(max_depth=10, random_state=0)
#   #model2 - GaussianNB()
#   #model2 = BaggingClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=0)
  
#   #model2 - GaussianNB

#   # model2 = GaussianNB()
#   # model2.fit(X_train,Y_train)
#   # print("model2: ",model2.score)
  
  #model2 - RandomForestClassifier
  # rfc=RandomForestClassifier(random_state=42)

  # param_grid = { 
  #     'n_estimators': [200, 500],
  #     'max_features': ['auto', 'sqrt', 'log2'],
  #     'max_depth' : [4,5,6,7,8],
  #     'criterion' :['gini', 'entropy']
  # }

  # CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
  # CV_rfc.fit(X_train, Y_train)
  # model2=CV_rfc
  # print("model2: ", model2.best_score_)

#   # #model3 - LinearSVC
#   pipe = Pipeline([
#       # the reduce_dim stage is populated by the param_grid
#       ('reduce_dim', 'passthrough'),
#       ('classify', LinearSVC(dual=False, max_iter=10000))
#   ])

#   N_FEATURES_OPTIONS = [2, 4, 8]
#   C_OPTIONS = [1, 10, 100, 1000]
#   param_grid = [
#       {
#           'reduce_dim': [PCA(iterated_power=7), NMF()],
#           'reduce_dim__n_components': N_FEATURES_OPTIONS,
#           'classify__C': C_OPTIONS
#       },
#       {
#           'reduce_dim': [SelectKBest(chi2)],
#           'reduce_dim__k': N_FEATURES_OPTIONS,
#           'classify__C': C_OPTIONS
#       },
#   ]
#   reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

#   grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
#   grid.fit(X_train, Y_train)
#   model3=grid
#   print("model3: ", model3.best_score_)

#   # GBC = GradientBoostingClassifier()
#   # gb_param_grid = {'loss' : ["deviance"],
#   #               'n_estimators' : [100,200,300],
#   #               'learning_rate': [0.1, 0.05, 0.01],
#   #               'max_depth': [4, 8],
#   #               'min_samples_leaf': [100,150],
#   #               'max_features': [0.3, 0.1] 
#   #               }

#   # gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, scoring="accuracy", n_jobs= 4, verbose = 1)

#   # gsGBC.fit(X_train,Y_train)
#   # model4=gsGBC
#   # print("model4: ",model4.score)

#   #model5 - ExtraTreesClassifier
#   ExtC = ExtraTreesClassifier()

#   ## Search grid for optimal parameters
#   ex_param_grid = {"max_depth": [None],
#                 "max_features": [1, 3, 10],
#                 "min_samples_split": [2, 3, 10],
#                 "min_samples_leaf": [1, 3, 10],
#                 "bootstrap": [False],
#                 "n_estimators" :[100,300],
#                 "criterion": ["gini"]}


#   gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, scoring="accuracy", n_jobs= 4, verbose = 1)

#   gsExtC.fit(X_train,Y_train)
#   model5= gsExtC
#   print("model5: ",model5.best_score_)
#   # ensemble = VotingClassifier(estimators=[('model1',model1),('model2',model2),('model3',model3),('model4',model4),('model5',model5)])
#   ensemble = VotingClassifier(estimators=[('model1',model1),('model3',model3),('model5',model5)])
#   ensemble.fit(X_train,Y_train)
#   #return ensemble

#   return ensemble